# Exercise 3 - Data Lake on S3

In [2]:
import findspark

In [3]:
findspark.init('/usr/local/Cellar/apache-spark/3.0.1/libexec')

In [4]:
from pyspark.sql import SparkSession
import os
import configparser

# Make sure that your AWS credentials are loaded as env vars

In [5]:
config = configparser.ConfigParser()

In [6]:
ls

4_data_wrangling.ipynb                bootstrap_emr.sh
7_data_wrangling-sql.ipynb            credentials.cfg
Exercise3_DataLakeonS3.ipynb          exercise2_AdvancedAnalyticsNLP.ipynb
Exercise_1_SchemaOnRead.ipynb         reddit-worldnews.json
Exercise_Creating_EMR_Clusters        spark-cluster.pem
NASA_access_log_Jul95.gz              spark-nlp-2.4.4/
README.md                             spark-warehouse/
Untitled.ipynb                        sparkify_log_small.json


In [7]:
#Normally this file should be in ~/.aws/credentials
config.read_file(open('credentials.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

# Create spark session with hadoop-aws package

In [13]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Load data from S3

In [22]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv")

In [20]:
df.printSchema()
df.show(5)

NameError: name 'df' is not defined

# Infer schema, fix header and separator

In [24]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv",sep=";", inferSchema=True, header=True)

In [ ]:
df.printSchema()
df.show(5)

# Fix the data yourself 

In [ ]:
import  pyspark.sql.functions as F
dfPayment = df.withColumn("payment_date", F.to_timestamp("payment_date"))
dfPayment.show(5)

# Extract the month

In [ ]:
dfPayment = dfPayment.withColumn("month", F.month("payment_date"))
dfPayment.show(5)

# Computer aggregate revenue per month

In [ ]:
dfPayment.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month, sum(amount) as revenue
    FROM payment
    GROUP BY month
    ORDER BY revenue desc
    """).show()

# Fix the schema

In [27]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date

In [29]:
paymentSchema = R([
    Fld("payment_id", Int()),
    Fld("customer_id", Int()),
    Fld("staff_id", Int()),
    Fld("rental_id", Int()),
    Fld("amount", Dbl()),
    Fld("payment_date", Date()),
])

In [32]:
dfPaymentWithSchema = spark.read.csv("s3a://udacity-dend/pagila/payment.csv", sep=";", schema = paymentSchema, header=True)

Py4JJavaError: An error occurred while calling o114.csv.
: java.lang.NumberFormatException: For input string: "100M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1538)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3303)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSourc

In [ ]:
dfPaymentWithSchema.printSchema()
df.show(5)

In [ ]:
dfPaymentWithSchema.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month(payment_date) as m, sum(amount) as revenue
    FROM payment
    GROUP BY m
    ORDER BY revenue DESC""").show()